## Домашнее задание №4. Работа с NPV моделью

*Дата выдачи: 28.01.2026*

*Дедлайн: 02.02.2026 23:59*

### Задание №1: проанализировать чувствительномть модели к параметрам

Формат -- меняем параметры вверх-вниз, смотрим на NPV и объясняем, почему это происходит.

### Задание №2: добавить страховку в NPV модель
- Страховка стоит 0.5% от лимита
- Подключена у 40% клиентов в статусе CUR и у 60% в статусе DLQ
- Возможно наступление страхового случая с вероятностью 0.1%, при подключенной страховке мы должны возместить всю сумму текущей задолженности

Вам нужно внести изменения в код ниже. Где – подсказывать не будем, это часть задания. Добавьте комментарий с указанием места, котороые вы поменяли.

### Класс модели

In [19]:
import numpy as np

class NPVModel:

    avg_missed_payments = 1.5
    recovery = 0.50
    dlq_penalty_amount = 500
    oper_costs = 100
    collection_costs = 600
    tax_rate = 0.20
    discounting_rate = 0.30
    eq_req = 0.125
    cost_of_funds = 0.16
    dlq_ratio = np.ones(101)*0.20
    dlq_ratio[0] = 0
    acquisition_cost = 1000
    ins_cost_rate = 0.05

    def __init__(self):
        pass

    def model_balance_calculations(self, amount, rate, term):
        '''
        Расчет модельных баланса, выплаченных процентов, регулярного платежа
        :param amount: Сумма кредита
        :param rate: Ставка
        :param term: Срок
        :return:
        balance : np.array(101) : остаток тела долга по кредиту на каждый месяц
        interest : np.array(101) : выплата по процентам каждый месяц
        regular_payment : float : размер регулярного платежа
        '''

        regular_payment = round(amount * (rate / 12) *\
                                (1 + (rate / 12)) / (1 - ((1 + rate / 12) ** (-term))))
        # График баланса и процентов
        balance = np.zeros(101)
        interest = np.zeros(101)
        balance[0] = amount

        for i in range(1, term + 1):
            int_payment = balance[i - 1] * rate / 12
            debt_payment = regular_payment - int_payment
            balance[i] = max(0, round(balance[i - 1] - debt_payment))
            interest[i] = int_payment
        return balance, interest, regular_payment

    def distribution_calc(self, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет распределения по статусам
        :param portfolio_distribution: график распределения по статусам
        :return:
        cur_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
        dlq_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
        act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
        def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
        clo_dist : np.array(101) : доля клиентов в статусе CLO на каждый месяц
        '''
        
        cur_dist = portfolio_distribution['CUR']
        dlq_dist = portfolio_distribution['DLQ']
        act_dist = cur_dist + dlq_dist
        def_dist = portfolio_distribution['DEF'] 
        clo_dist = portfolio_distribution['CLO']


        return cur_dist, dlq_dist, act_dist, def_dist, clo_dist

    def cur_balance_calc(self, model_balance, cur_dist):
        '''
        Расчет модельного баланса в статусе CUR
        :param model_balance: плановый график баланса
        :param cur_dist: доля клиентов в статусе CUR на каждый месяц
        :return:
        principal_balance_cur : np.array(101) : principal balance в статусе CUR
        gross_balance_cur : np.array(101) : gross balance в статусе CUR
        '''
        principal_balance_cur = model_balance*cur_dist
        gross_balance_cur = model_balance*cur_dist
        return principal_balance_cur, gross_balance_cur

    def dlq_balance_calc(self, model_balance, regular_payment, dlq_dist):
        '''
        Расчет модельного баланса в статусе DLQ
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_dlq : np.array(101) : principal balance в статусе DLQ
        gross_balance_dlq : np.array(101) : gross balance в статусе DLQ
        '''
        principal_balance_dlq = np.append(0, model_balance[:-1])*dlq_dist
        gross_balance_dlq = (model_balance + regular_payment*self.avg_missed_payments)*dlq_dist
        return principal_balance_dlq, gross_balance_dlq

    def act_balance_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq):
        '''
        Расчет модельного баланса в статусе ACT
        :param principal_balance_cur: principal balance в статусе CUR
        :param principal_balance_dlq: principal balance в статусе DLQ
        :param gross_balance_cur: principal balance в статусе CUR
        :param gross_balance_dlq: principal balance в статусе DLQ
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_act : np.array(101) : principal balance в статусе ACT
        gross_balance_act : np.array(101) : gross balance в статусе ACT
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq
        gross_balance_act = gross_balance_cur + gross_balance_dlq
        return principal_balance_act, gross_balance_act

    def def_balance_calc(self, model_balance, regular_payment, def_dist):
        '''
        Расчет модельного баланса в статусе DEF
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        principal_balance_def : np.array(101) : principal balance в статусе DEF
        gross_balance_def : np.array(101) : gross balance в статусе DEF
        '''
        principal_balance_def = np.zeros(101)
        gross_balance_def = np.zeros(101)

        def_dist_change = def_dist[4:] - def_dist[3:-1]
        principal_balance_def[4:] = np.cumsum(model_balance[:-4] * def_dist_change)
        gross_balance_def[4:] = np.cumsum((model_balance[4:] + 4 * regular_payment) * def_dist_change)

        return principal_balance_def, gross_balance_def

    def profit_calc(self, principal_balance_act, principal_balance_def, term, rate, dlq_dist):
        '''

        :param principal_balance_act: principal balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param rate: тавка по кредиту
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        profit : np.array(101) : доход на каждый месяц
        '''
        interest_profit = principal_balance_act*rate/12
        penatly_profit = dlq_dist*self.dlq_penalty_amount

        new_def_balance = np.append(principal_balance_def[1:], 0) - principal_balance_def
        recovery_profit = new_def_balance*self.recovery

        profit = interest_profit + recovery_profit + penatly_profit
        profit[term+1:] = 0

        return profit

    def loss_calc(self, gross_balance_act, principal_balance_def, term, act_dist, dlq_dist, def_dist):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param act_dist: доля клиентов в статусе ACT на каждый месяц
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        loss : np.array(101) : лосс на каждый месяц
        '''
        oper_loss = np.append(act_dist[1:], 0)*self.oper_costs
        loan_loss =  principal_balance_def - np.append(0, principal_balance_def[:-1])
        collection_loss = (np.append(dlq_dist[1:], 0) + np.append(def_dist[1:], 0))*self.collection_costs

        
        prev_gross_balance_act = np.append(gross_balance_act[0], gross_balance_act[:-1]) 
        cost_of_funds_loss = prev_gross_balance_act*(1 - self.eq_req)*self.cost_of_funds/12

        loss = loan_loss + cost_of_funds_loss + oper_loss + collection_loss
        loss[term+1:] = 0

        return loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss

    def assets_liabilities_calc(self, gross_balance_act):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :return:
        assets : np.array(101) : активы в проекте
        eq_req_curve : np.array(101) : активы, обеспеченные капиталом
        fund_req_curve : np.array(101) : активы, обеспеченные фондами
        '''

        # assets
        assets = gross_balance_act

        #liabilities
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)

        return assets, eq_req_curve, fund_req_curve

    def niat_calc(self, profit, loss):
        '''
        Расчет NIAT
        :param profit: суммарный доход на каждый месяц
        :param loss: суммарный лосс на каждый месяц
        :return:
        nibt : np.array(101) : прибыль до налогообложения
        niat : np.array(101) : прибыль после налогообложения
        tax : np.array(101) : налог в каждом месяце
        '''

        nibt = profit - loss
        tax = nibt*self.tax_rate
        niat = nibt - tax

        return nibt, niat, tax

    def cashflow_calc(self, principal_balance_act, principal_balance_def, amount, profit,
                      cost_of_funds_loss, oper_costs, collection_costs, tax, niat, eq_req_curve, fund_req_curve):
        '''
        Расчет денежных потоков
        :param principal_balance_act: principal balance в статусе DEF
        :param principal_balance_def: principal balance в статусе DEF
        :param amount: срок кредита
        :param profit: доход на каждый месяц
        :param cost_of_funds_loss: лоссы на фондирование
        :param oper_costs: операционные расходы
        :param collection_costs: расходы на коллекшн
        :param tax: налог в каждом месяце
        :param niat: доход после налогообложения
        :param eq_req_curve: активы, обеспеченные капиталом
        :param fund_req_curve: активы, обеспеченные фондами
        :return:
        cf_to_client : np.array(101) : денежный поток к клиенту
        cf_to_shareholders : np.array(101) : денежный поток к акционерам
        cf_to_debtholders : np.array(101) : денежный поток к фондам
        cf_to_cost_and_tax : np.array(101) : денежный поток на косты и налоги
        '''
        # client
        delta_principal_balance_act = (np.append(0, principal_balance_act[:100]) - principal_balance_act)
        delta_principal_balance_def = (np.append(0, principal_balance_def[:100]) - principal_balance_def)
        repayments = delta_principal_balance_act - delta_principal_balance_def + profit
        cf_to_client = np.append(amount, -repayments[1:])

        # debtoholder
        fund_req_ch = fund_req_curve - np.append(fund_req_curve[0], fund_req_curve[:-1])
        fund_req_change = np.append(fund_req_curve[0], fund_req_ch[1:])
        cf_to_debtholders = cost_of_funds_loss - fund_req_change

        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

        # cost and tax
        cf_to_cost_and_tax = oper_costs + collection_costs + tax

        return cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax

    def npv_calc(self, amount, rate, term, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет NPV
        :param amount: сумма кредита
        :param rate: ставка по кредиту
        :param term: срок кредита
        :param portfolio_distribution: график распределения по статусам
        :param pd: вероятность дефолта к 12-ому месяцу
        :param pa: вероятность полного досрочного погашения к 6-ому месяцу
        :return: npv: чистая приведенная стоимость кредита
        '''

        # Считаем балансы
        model_balance, interest, regular_payment =  self.model_balance_calculations(amount, rate, term)

        # Считаем распределение
        cur_dist, dlq_dist, act_dist, def_dist, clo_dist = self.distribution_calc(portfolio_distribution)

        # Считаем балансы
        principal_balance_cur, gross_balance_cur = self.cur_balance_calc(model_balance, cur_dist)
        principal_balance_dlq, gross_balance_dlq = self.dlq_balance_calc(model_balance, regular_payment, dlq_dist)
        principal_balance_act, gross_balance_act = self.act_balance_calc(principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq)
        principal_balance_def, gross_balance_def = self.def_balance_calc(model_balance, regular_payment, def_dist)

        # Считаем профиты
        profit = self.profit_calc(principal_balance_act, principal_balance_def, term, rate, dlq_dist)
        loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss = self.loss_calc(gross_balance_act, principal_balance_def,
                                                                                                                  term, act_dist, dlq_dist, def_dist)
        nibt, niat, tax = self.niat_calc(profit, loss)

        # Считаем активы и денежные потоки
        assets, eq_req_curve, fund_req_curve = self.assets_liabilities_calc(gross_balance_act)
        cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax =\
            self.cashflow_calc(principal_balance_act, principal_balance_def, amount, profit, cost_of_funds_loss, oper_loss, collection_loss,
                               tax, niat, eq_req_curve, fund_req_curve)

        # Считаем NPV
        disc_curve = np.array([1 / ((1 + self.discounting_rate) ** (i / 12.)) for i in range(101)])
        pv = np.round(np.sum(disc_curve * cf_to_shareholders)).astype(int)
        npv = pv - self.acquisition_cost

        return npv

### Пример использования модели

In [20]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

In [21]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame({
    'CUR': cur_dist,
    'DLQ': dlq_dist,
    'DEF': def_dist,
    'CLO': clo_dist
})

df = df.reset_index().rename(columns={'index': 'Номер выписки'})
df = df.melt(id_vars='Номер выписки', value_vars=['DEF', 'DLQ', 'CUR', 'CLO'], 
             var_name='Status', value_name='Доля')

fig = px.area(df, x='Номер выписки', y='Доля', color='Status', 
              title='Распределение статусов по номерам выписки',
              color_discrete_map={
                  'DEF': '#D62728',
                  'DLQ': '#FF7F0E',
                  'CUR': '#2CA02C',
                  'CLO': '#1F77B4',
              },
              category_orders={'Status': ['DEF', 'DLQ', 'CUR', 'CLO']}
             )

fig.update_layout(height=600)
fig.update_xaxes(range=[0, 15])

fig.show()

In [22]:
model = NPVModel()
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(-3060)

In [23]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

dlq_ratio_array = np.ones(101) * 0.20
dlq_ratio_array[0] = 0  

model = NPVModel()

"""model.avg_missed_payments = 1.5
model.recovery = 0.50
model.dlq_penalty_amount = 500
model.oper_costs = 100
model.collection_costs = 600
model.tax_rate = 0.20
model.discounting_rate = 0.30  
model.eq_req = 0.125
model.cost_of_funds = 0.16     
model.dlq_ratio = dlq_ratio_array  
model.acquisition_cost = 1000
"""
model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(-3060)

In [24]:
print('увеличив долю дефолтного долга, которую банк может вернуть npv увеличится')
model.recovery = 0.60
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('операционные затраты растут -> npv уменьшается')
model.oper_costs = 120
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('налоги растут -> npv уменьшается')
model.tax_rate = 0.22
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('ставка дисконтирования падает -> npv увеличивается, тк будущие деньги становятся более ценными сегодня')
model.discounting_rate = 0.25  
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('доля кредитов обеспеченных собственным капиталом уменьшается -> npv увеличивается, тк в данной модели собственный капитал бесплаьный')
model.eq_req = 0.120 
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('стоимость привлечения средств уменьшается -> npv увеличивается')
model.cost_of_funds = 0.10
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")


dlq_ratio_array = np.ones(101) * 0.11
dlq_ratio_array[0] = 0  


print('количество клиентов в просрочке в каждый месяц уменьшается -> npv не меняется ПОТОМУ ЧТО \
      В profit_calc используется dlq_dist из распределения портфеля, а НЕ self.dlq_ratio из атрибутов класса')
model.dlq_ratio = dlq_ratio_array  
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")


cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.10

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

print('количество клиентов в просрочке в каждый месяц увеличивается -> npv падает, \
      так как часть переходит в дефолт, расходы на на коллекторскую едятельность увеличиваются,\
      хотя доходы от штрафов растут, они не покрывают расходы на коллект\
      а еще модель считает, что DLQ клиенты платят проценты')
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

print('стоимость привлечения клиента увеличивается -> npv уменьшается')
model.acquisition_cost = 5000
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")

увеличив долю дефолтного долга, которую банк может вернуть npv увеличится
NPV: -1001

операционные затраты растут -> npv уменьшается
NPV: -1135

налоги растут -> npv уменьшается
NPV: -1322

ставка дисконтирования падает -> npv увеличивается, тк будущие деньги становятся более ценными сегодня
NPV: -401

доля кредитов обеспеченных собственным капиталом уменьшается -> npv увеличивается, тк в данной модели собственный капитал бесплаьный
NPV: -310

стоимость привлечения средств уменьшается -> npv увеличивается
NPV: 10992

количество клиентов в просрочке в каждый месяц уменьшается -> npv не меняется ПОТОМУ ЧТО       В profit_calc используется dlq_dist из распределения портфеля, а НЕ self.dlq_ratio из атрибутов класса
NPV: 10992

количество клиентов в просрочке в каждый месяц увеличивается -> npv падает,       так как часть переходит в дефолт, расходы на на коллекторскую едятельность увеличиваются,      хотя доходы от штрафов растут, они не покрывают расходы на коллект      а еще модель счита

In [25]:
clo_dist[6:] += 0.10
cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }
print('увеличение клиентов в статусе clo -> npv падает, так как cur соответственно уменьшается и interest_profit меньше')
npv_result = model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)
print(f"NPV: {npv_result}\n")




увеличение клиентов в статусе clo -> npv падает, так как cur соответственно уменьшается и interest_profit меньше
NPV: 4295



### Место для комментариев

- Возможно наступление страхового случая с вероятностью 0.1%, при подключенной страховке мы должны возместить всю сумму текущей задолженности

In [26]:
import numpy as np

class NPVModel:

    avg_missed_payments = 1.5
    recovery = 0.50
    dlq_penalty_amount = 500
    oper_costs = 100
    collection_costs = 600
    tax_rate = 0.20
    discounting_rate = 0.30
    eq_req = 0.125
    cost_of_funds = 0.16
    dlq_ratio = np.ones(101)*0.20
    dlq_ratio[0] = 0
    acquisition_cost = 1000
    ins_cost_rate = 0.005 #цена страховки от лимита 
    ins_cur = 0.4 # процент cur со страховкой
    ins_dlq = 0.6 # процент dlq со страховкой
    ins_prob = 0.001 # вероятность наступления стархового случая

    def __init__(self):
        pass

    def model_balance_calculations(self, amount, rate, term):
        '''
        Расчет модельных баланса, выплаченных процентов, регулярного платежа
        :param amount: Сумма кредита
        :param rate: Ставка
        :param term: Срок
        :return:
        balance : np.array(101) : остаток тела долга по кредиту на каждый месяц
        interest : np.array(101) : выплата по процентам каждый месяц
        regular_payment : float : размер регулярного платежа
        '''

        regular_payment = round(amount * (rate / 12) *\
                                (1 + (rate / 12)) / (1 - ((1 + rate / 12) ** (-term))))
        # График баланса и процентов
        balance = np.zeros(101)
        interest = np.zeros(101)
        balance[0] = amount

        for i in range(1, term + 1):
            int_payment = balance[i - 1] * rate / 12
            debt_payment = regular_payment - int_payment
            balance[i] = max(0, round(balance[i - 1] - debt_payment))
            interest[i] = int_payment
        return balance, interest, regular_payment

    def distribution_calc(self, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет распределения по статусам
        :param portfolio_distribution: график распределения по статусам
        :return:
        cur_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
        dlq_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
        act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
        def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
        clo_dist : np.array(101) : доля клиентов в статусе CLO на каждый месяц
        '''
        
        cur_dist = portfolio_distribution['CUR']
        dlq_dist = portfolio_distribution['DLQ']
        act_dist = cur_dist + dlq_dist
        def_dist = portfolio_distribution['DEF'] 
        clo_dist = portfolio_distribution['CLO']


        return cur_dist, dlq_dist, act_dist, def_dist, clo_dist

    def cur_balance_calc(self, model_balance, cur_dist):
        '''
        Расчет модельного баланса в статусе CUR
        :param model_balance: плановый график баланса
        :param cur_dist: доля клиентов в статусе CUR на каждый месяц
        :return:
        principal_balance_cur : np.array(101) : principal balance в статусе CUR
        gross_balance_cur : np.array(101) : gross balance в статусе CUR
        '''
        principal_balance_cur = model_balance*cur_dist
        gross_balance_cur = model_balance*cur_dist
        return principal_balance_cur, gross_balance_cur

    def dlq_balance_calc(self, model_balance, regular_payment, dlq_dist):
        '''
        Расчет модельного баланса в статусе DLQ
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_dlq : np.array(101) : principal balance в статусе DLQ
        gross_balance_dlq : np.array(101) : gross balance в статусе DLQ
        '''
        principal_balance_dlq = np.append(0, model_balance[:-1])*dlq_dist
        gross_balance_dlq = (model_balance + regular_payment*self.avg_missed_payments)*dlq_dist
        return principal_balance_dlq, gross_balance_dlq

    def act_balance_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq):
        '''
        Расчет модельного баланса в статусе ACT
        :param principal_balance_cur: principal balance в статусе CUR
        :param principal_balance_dlq: principal balance в статусе DLQ
        :param gross_balance_cur: principal balance в статусе CUR
        :param gross_balance_dlq: principal balance в статусе DLQ
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_act : np.array(101) : principal balance в статусе ACT
        gross_balance_act : np.array(101) : gross balance в статусе ACT
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq
        gross_balance_act = gross_balance_cur + gross_balance_dlq
        return principal_balance_act, gross_balance_act

    def def_balance_calc(self, model_balance, regular_payment, def_dist):
        '''
        Расчет модельного баланса в статусе DEF
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        principal_balance_def : np.array(101) : principal balance в статусе DEF
        gross_balance_def : np.array(101) : gross balance в статусе DEF
        '''
        principal_balance_def = np.zeros(101)
        gross_balance_def = np.zeros(101)

        def_dist_change = def_dist[4:] - def_dist[3:-1]
        principal_balance_def[4:] = np.cumsum(model_balance[:-4] * def_dist_change)
        gross_balance_def[4:] = np.cumsum((model_balance[4:] + 4 * regular_payment) * def_dist_change)

        return principal_balance_def, gross_balance_def

    def profit_calc(self, principal_balance_cur, principal_balance_dlq, principal_balance_def, term, rate, dlq_dist):
        '''

        :param principal_balance_act: principal balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param rate: тавка по кредиту
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        profit : np.array(101) : доход на каждый месяц
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq # тут поменял и в передаче

        interest_profit = principal_balance_act*rate/12
        penatly_profit = dlq_dist*self.dlq_penalty_amount

        new_def_balance = np.append(principal_balance_def[1:], 0) - principal_balance_def
        recovery_profit = new_def_balance*self.recovery



        insurance_limit_cur = principal_balance_dlq[0] * self.ins_dlq #
        insurance_limit_dlq = principal_balance_cur[0] * self.ins_cur #

        insurance_profit_monthly = np.zeros(101)
        insurance_profit_monthly[:term+1] = (insurance_limit_cur + insurance_limit_dlq) * self.ins_cost_rate

        profit = interest_profit + recovery_profit + penatly_profit + insurance_profit_monthly#
        profit[term+1:] = 0

        return profit

    def loss_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_act, principal_balance_def, term, act_dist, dlq_dist, def_dist):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param act_dist: доля клиентов в статусе ACT на каждый месяц
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        loss : np.array(101) : лосс на каждый месяц
        '''

        oper_loss = np.append(act_dist[1:], 0)*self.oper_costs
        loan_loss =  principal_balance_def - np.append(0, principal_balance_def[:-1])
        collection_loss = (np.append(dlq_dist[1:], 0) + np.append(def_dist[1:], 0))*self.collection_costs
        
        insurance_loss = np.zeros(101)
        insurance_limit_cur = principal_balance_dlq[0] * self.ins_cur
        insurance_limit_dlq = principal_balance_cur[0] * self.ins_dlq
        for t in range(1, term + 1):
                # Вероятность, что страховой случай произойдет именно в месяц t
                # (не произошел в предыдущие месяцы) × вероятность в текущем месяце
                prob_in_month_t = self.ins_prob * ((1 - self.ins_prob) ** (t - 1))
                
                # Ожидаемая выплата в месяц t
                insurance_loss[t] = (
                    insurance_limit_cur * prob_in_month_t +
                    insurance_limit_dlq * prob_in_month_t
                )
        

        prev_gross_balance_act = np.append(gross_balance_act[0], gross_balance_act[:-1]) 
        cost_of_funds_loss = prev_gross_balance_act*(1 - self.eq_req)*self.cost_of_funds/12

        loss = loan_loss + cost_of_funds_loss + oper_loss + collection_loss + insurance_loss
        loss[term+1:] = 0

        return loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss, insurance_loss

    def assets_liabilities_calc(self, gross_balance_act, insurance_loss):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :return:
        assets : np.array(101) : активы в проекте
        eq_req_curve : np.array(101) : активы, обеспеченные капиталом
        fund_req_curve : np.array(101) : активы, обеспеченные фондами
        '''

        # assets
        assets = gross_balance_act - insurance_loss 

        #liabilities
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)

        return assets, eq_req_curve, fund_req_curve

    def niat_calc(self, profit, loss):
        '''
        Расчет NIAT
        :param profit: суммарный доход на каждый месяц
        :param loss: суммарный лосс на каждый месяц
        :return:
        nibt : np.array(101) : прибыль до налогообложения
        niat : np.array(101) : прибыль после налогообложения
        tax : np.array(101) : налог в каждом месяце
        '''

        nibt = profit - loss
        tax = nibt*self.tax_rate
        niat = nibt - tax

        return nibt, niat, tax

    def cashflow_calc(self, principal_balance_act, principal_balance_def, amount, profit,
                      cost_of_funds_loss, oper_costs, collection_costs, tax, niat, eq_req_curve, fund_req_curve):
        '''
        Расчет денежных потоков
        :param principal_balance_act: principal balance в статусе DEF
        :param principal_balance_def: principal balance в статусе DEF
        :param amount: срок кредита
        :param profit: доход на каждый месяц
        :param cost_of_funds_loss: лоссы на фондирование
        :param oper_costs: операционные расходы
        :param collection_costs: расходы на коллекшн
        :param tax: налог в каждом месяце
        :param niat: доход после налогообложения
        :param eq_req_curve: активы, обеспеченные капиталом
        :param fund_req_curve: активы, обеспеченные фондами
        :return:
        cf_to_client : np.array(101) : денежный поток к клиенту
        cf_to_shareholders : np.array(101) : денежный поток к акционерам
        cf_to_debtholders : np.array(101) : денежный поток к фондам
        cf_to_cost_and_tax : np.array(101) : денежный поток на косты и налоги
        '''
        # client
        delta_principal_balance_act = (np.append(0, principal_balance_act[:100]) - principal_balance_act)
        delta_principal_balance_def = (np.append(0, principal_balance_def[:100]) - principal_balance_def)
        repayments = delta_principal_balance_act - delta_principal_balance_def + profit
        cf_to_client = np.append(amount, -repayments[1:])

        # debtoholder
        fund_req_ch = fund_req_curve - np.append(fund_req_curve[0], fund_req_curve[:-1])
        fund_req_change = np.append(fund_req_curve[0], fund_req_ch[1:])
        cf_to_debtholders = cost_of_funds_loss - fund_req_change

        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

        # cost and tax
        cf_to_cost_and_tax = oper_costs + collection_costs + tax

        return cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax

    def npv_calc(self, amount, rate, term, portfolio_distribution, pd=None, pa=None):
        '''
        Расчет NPV
        :param amount: сумма кредита
        :param rate: ставка по кредиту
        :param term: срок кредита
        :param portfolio_distribution: график распределения по статусам
        :param pd: вероятность дефолта к 12-ому месяцу
        :param pa: вероятность полного досрочного погашения к 6-ому месяцу
        :return: npv: чистая приведенная стоимость кредита
        '''

        # Считаем балансы
        model_balance, interest, regular_payment =  self.model_balance_calculations(amount, rate, term)

        # Считаем распределение
        cur_dist, dlq_dist, act_dist, def_dist, clo_dist = self.distribution_calc(portfolio_distribution)

        # Считаем балансы
        principal_balance_cur, gross_balance_cur = self.cur_balance_calc(model_balance, cur_dist)
        principal_balance_dlq, gross_balance_dlq = self.dlq_balance_calc(model_balance, regular_payment, dlq_dist)
        principal_balance_act, gross_balance_act = self.act_balance_calc(principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq)
        principal_balance_def, gross_balance_def = self.def_balance_calc(model_balance, regular_payment, def_dist)

        # Считаем профиты
        profit = self.profit_calc(principal_balance_cur, principal_balance_dlq, principal_balance_def, term, rate, dlq_dist)
        loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss, insurance_loss = self.loss_calc(principal_balance_cur, principal_balance_dlq, gross_balance_act, principal_balance_def,
                                                                                                                  term, act_dist, dlq_dist, def_dist)
        nibt, niat, tax = self.niat_calc(profit, loss)

        # Считаем активы и денежные потоки
        assets, eq_req_curve, fund_req_curve = self.assets_liabilities_calc(gross_balance_act, insurance_loss)
        cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax =\
            self.cashflow_calc(principal_balance_act, principal_balance_def, amount, profit, cost_of_funds_loss, oper_loss, collection_loss,
                               tax, niat, eq_req_curve, fund_req_curve)

        # Считаем NPV
        disc_curve = np.array([1 / ((1 + self.discounting_rate) ** (i / 12.)) for i in range(101)])
        pv = np.round(np.sum(disc_curve * cf_to_shareholders)).astype(int)
        npv = pv - self.acquisition_cost

        return npv

In [27]:
cur_dist = np.zeros(101)
dlq_dist = np.zeros(101)
clo_dist = np.zeros(101)
def_dist = np.zeros(101)

dlq_dist[1:12] += 0.05

def_dist[6:] += 0.03
def_dist[9:] += 0.03

def_dist[12:] += dlq_dist[11]

clo_dist[3:] += 0.10
clo_dist[6:] += 0.10
clo_dist[12:] = 1 - def_dist[12]

cur_dist = 1 - def_dist - clo_dist - dlq_dist

portfolio_distribution = {
    'CUR' : cur_dist,
    'DLQ' : dlq_dist,
    'DEF' : def_dist,
    'CLO' : clo_dist
         }

dlq_ratio_array = np.ones(101) * 0.20
dlq_ratio_array[0] = 0  

model = NPVModel()

model.npv_calc(amount=500_000, term=12, rate=0.25, portfolio_distribution=portfolio_distribution)

np.int64(3608)

## что я поменял
добавил атрибуты класса, изменил profit_calc (insurance_profit), loss_calc (insurance_loss). Из активов (assets) вычел insurance_loss